In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

### 読み込み

In [ ]:
df_all = pd.read_csv("/kaggle/input/nfl-big-data-bowl-2020/train.csv")

In [ ]:
df_copy = df_all.copy()

In [ ]:
df_all.head()

In [ ]:
df_all.columns

## データ確認

In [ ]:
df_all[['GameId', 'PlayId', 'Team', 'X', 'Y', 'S', 'A',
        'Dis', 'Orientation', 'Dir', 'NflId', 'DisplayName', 'JerseyNumber']].head()

In [ ]:
df_all[['Season', 'YardLine', 'Quarter', 'GameClock', 
        'PossessionTeam', 'Down', 'Distance', 'FieldPosition']].head()

In [ ]:
df_all[['HomeScoreBeforePlay', 'VisitorScoreBeforePlay',
       'NflIdRusher', 'OffenseFormation', 'OffensePersonnel',
       'DefendersInTheBox', 'DefensePersonnel']].head()

In [ ]:
df_all[['PlayDirection', 'TimeHandoff','TimeSnap', 'Yards']].head()

In [ ]:
df_all[['PlayerHeight', 'PlayerWeight', 'PlayerBirthDate',
       'PlayerCollegeName', 'Position', 'HomeTeamAbbr', 'VisitorTeamAbbr']].head()

In [ ]:
df_all[['Week', 'Stadium', 'Location', 'StadiumType', 'Turf', 'GameWeather',
       'Temperature', 'Humidity', 'WindSpeed', 'WindDirection']].head()

In [ ]:
# 選手一人毎の情報がある列
# 他の列は同一PlayId内では同じ
personal_columms = ['X', 'Y', 'S', 'A', 'Dis', 'Orientation', 'Dir', 
                    'NflId', 'DisplayName', 'JerseyNumber', 
                   'PlayerHeight', 'PlayerWeight', 'PlayerBirthDate','PlayerCollegeName', 'Position']

In [ ]:
df_all.shape

In [ ]:
pd.set_option("display.max_columns", 80)

In [ ]:
df_all.describe()

# 前処理

### チーム名確認

In [ ]:
# チーム名は２年とも同じ
print(sorted(df_all[df_all["Season"]==2017]['PossessionTeam'].unique()))
print(sorted(df_all[df_all["Season"]==2018]['PossessionTeam'].unique()))

In [ ]:
# 列によりチーム名の表記揺れ
print(sorted(df_all['PossessionTeam'].unique()))
print(sorted(df_all['HomeTeamAbbr'].unique()))
print(sorted(df_all['VisitorTeamAbbr'].unique()))


In [ ]:
df_all['FieldPosition'].unique()
# 50 yards にいる時は nan

In [ ]:
# wikipedia にあるチーム名
# ["ARI", "ATL", "BAL", "BUF", "CAR", "CHI", "CIN", "CLE", "DAL", "DEN", "DET", "GB","HOU", "IND", "JAX", 
# "KC", "LAC", "LAR", "MIA", "MIN", "NE", "NO", "NYG", "NYJ", "OAK", "PHI", "PIT", "SEA", "SF", "TB",
# "TEN", "WAS" ]

# ARZ=ARI, BLT=BAL, CLV=CLE, HST=HOU

In [ ]:
# 風速の表記いろいろ
df_all["WindSpeed"].unique()

In [ ]:
import re

In [ ]:
def is_float(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

In [ ]:
# 芝の種類の表記いろいろ
df_all["Turf"].unique()

In [ ]:
def preprocess(df):
#    df=pd.get_dummies(df,columns=['Team','PlayDirection','OffenseFormation','Position'])
    df=pd.get_dummies(df,columns=['OffenseFormation'], drop_first=True)
    df=df[(df['StadiumType']!='Cloudy') & (df['StadiumType']!='Bowl')]
#    df=df.drop('FieldPosition', axis=1)
    
    # SadiumType:屋内・屋外の２分類に
    outdoor=['Outdoor', 'Outdoors','Open','Indoor, Open Roof','Outdoor Retr Roof-Open', 'Oudoor',
             'Ourdoor','Retr. Roof-Open','Outdor',
               'Retr. Roof - Open', 'Domed, Open', 'Domed, open', 'Outside','Heinz Field']
    indoor=['Indoors', 'RetractableRoof', 'Indoor','Retr. Roof-Closed','Dome', 'Domed, closed',
            'Indoor, Roof Closed', 'Retr. Roof Closed','Closed Dome','Dome, closed','Domed']
    df['stadiumtype']=(df['StadiumType'].isin(outdoor)*1)
    
    # 天候：雨か否か
    rain=['Light Rain', 'Showers','Cloudy with periods of rain, thunder possible. Winds shifting to WNW,10-20 mph.',
          'Rain', 'Heavy lake effect snow','Snow', 'Cloudy, Rain','Rain shower','Rainy']
    df['weather']=(~df['GameWeather'].isin(rain)*1)
    
    # 芝：天然か否か
    natural_turf = ['Grass', 'Natural Grass', 'Artificial', 'Natural grass', 'grass', 
        'Natural', 'Naturall Grass',  'natural grass']
    df_all["natural_turf"] = df_all["Turf"].isin(natural_turf)
    
    # 身長(feet-inch)、時計：floatに
    df['PlayerHeight']= df['PlayerHeight'].apply(lambda x: 12*int(x.split('-')[0])+int(x.split('-')[1]))
    df['gameclock']=[ pd.Timedelta(val).total_seconds() for val in df['GameClock']]
    
    # チーム名の表記揺れ修正
    df = df.replace({'ARZ':'ARI', 'BLT':'BAL', 'CLV':'CLE', 'HST':'HOU'})
    
    # 風速を数値に
#    df["WindSpeed"] = df["WindSpeed"].replace({ 'SSW':np.nan, '11-17':14, "14-23":18.5, '13 MPH':13, 
#                             '12-22':17, '4 MPh':4, '15 gusts up to 25':15, '10MPH':10, '10mph':10,
#                             'E':np.nan, '7 MPH':7, 'Calm':0, '6 mph':6, 'SE':np.nan, '10-20':15})
#    df["WindSpeed"] = df["WindSpeed"].astype(float)
    df["WindSpeed"] = df["WindSpeed"].apply(lambda value : 
                            np.mean([ float(oo) for oo in re.split('[^0-9.]+', str(value)) if is_float(oo) ]))
    # 屋内は無風のはず
    df.loc[ df['StadiumType'].isin(indoor), "WindSpeed" ] == 0

    return df

In [ ]:
df_all = preprocess(df_all)

In [ ]:
df_all["WindSpeed"].unique()

In [ ]:
# ボールを持っている選手の行のみを選択
df_play = df_all[df_all["NflId"]==df_all["NflIdRusher"]].copy()

In [ ]:
# ボールを持つのはほぼRB
df_play["Position"].value_counts()

# 守備選手（DT, DE）が持っているのはどういうケース？

In [ ]:
df_play["Yards"].hist(range=(-20,50),bins=70)

# 追加の特徴量

## ポジションの分類

In [ ]:
df_all["Position"].unique()

In [ ]:
# オフェンスチーム
# QB
# RB（FB、HB、TB）
# WR（SE、FL、SB、WB）
# TE
# OL（C、G、T、E）
# ディフェンスチーム
# DL（DT（NT）、DE）
# LB（ILB（MLB）、OLB（LOLB、ROLB、SLB、WLB））
# DB（CB、S（SS、FS））
# スペシャルチーム
# K、P、LS、H、KR/PR

offence_position = ['WR', 'TE', 'T', 'QB', 'RB', 'G', 'C', 'FB', 'HB',  'OT', 'OG', ]
#OL_position = ['T', 'G', 'C', 'OT', 'OG']
#RB_position = ['RB', 'FB', 'HB']

defence_position = ['SS', 'DE', 'ILB', 'FS', 'CB', 'DT', 'OLB', 'NT', 'MLB', 'LB', 'S', 'DL', 'DB', 'SAF']
#DL_position = ['DL', 'DT', 'DE']
#LB_position = ['LB', 'ILB', ]

In [ ]:
df_all.loc[:, "offence"] = 0
df_all.loc[df_all["Position"].isin(offence_position), "offence"] = 1

## ポジション毎の人数

In [ ]:
# position_count=df_all.groupby(["PlayId", "Position"]).count()

In [ ]:
# df_position = position_count["GameId"].unstack().fillna(0).astype(int)

In [ ]:
def count_position(df_play):
    # 文字列　→ { "ポジション":人数 } の辞書
    df_play.loc[:, "OffensePersonnel"] = df_play["OffensePersonnel"].apply(
        lambda x : { i.split(" ")[-1]:int(i.split(" ")[-2]) for i in x.split(",")} )
    df_play.loc[:, "DefensePersonnel"] = df_play["DefensePersonnel"].apply(
        lambda x : { i.split(" ")[-1]:int(i.split(" ")[-2]) for i in x.split(",")} )

    # ポジション毎の人数
    for position in ["DL", "LB", "DB"]:
        df_play.loc[:, position] = [ d[position] for d in df_play["DefensePersonnel"]]
    for position in ["RB", "TE", "WR"]:
        df_play.loc[:, position] = [ d[position] for d in df_play["OffensePersonnel"]]

    return df_play

In [ ]:
df_play = count_position(df_play)

## 選手・チームの平均情報追加

In [ ]:
def team_name(df_play):
    # rusher選手の所属チーム名
    df_play.loc[:, "PlayerTeamAbbr"] = df_play["HomeTeamAbbr"]
    df_play.loc[ df_play["Team"]=="away", "PlayerTeamAbbr"] = df_play["VisitorTeamAbbr"]
    
    # defence側のチーム名
    df_play.loc[:, "DefenceTeamAbbr"] = df_play["HomeTeamAbbr"]
    df_play.loc[ df_play["Team"]=="home", "DefenceTeamAbbr"] = df_play["VisitorTeamAbbr"]
    
    return df_play

In [ ]:
# df_play.groupby(["Season", "Team", "PlayerTeamAbbr"]).std()[["Yards"]].hist()

In [ ]:
def average_yards(df_play):

    # チーム毎の平均獲得ヤード（home/away別, 年度別）
    team_yards_df = df_play.groupby(["Season", "Team", "PlayerTeamAbbr"]).mean()[["Yards"]]
    team_yards_df = team_yards_df.unstack(level=0)["Yards"]

    
    # チーム毎の平均損失ヤード（home/away別, 年度別）
    # "Team" に入っているのは攻撃側(rusher)がhomeかawayか
    defence_yards_df = df_play.groupby(["Season", "Team", "DefenceTeamAbbr"]).mean()[["Yards"]]
    defence_yards_df = defence_yards_df.unstack(level=0)["Yards"]
        
    # 選手毎の平均獲得ヤード
    personal_yards = df_play.groupby(["Season", "NflId"]).agg(["mean", "std", "count"])["Yards"]
    personal_yards = personal_yards.unstack(level=0)
    
    # 新人の前年成績には平均値を入れておく
    personal_yards.fillna({'mean': personal_yards["mean"].mean(),
                                            'std': personal_yards["std"].mean(), 'count': 0}, inplace=True)
    personal_yards = personal_yards.swaplevel(0, 1, axis=1)
    
    return team_yards_df, defence_yards_df, personal_yards

In [ ]:
df_play = team_name(df_play)
team_yards_df, defence_yards_df, personal_yards = average_yards(df_play)

In [ ]:
team_yards_df.head()

In [ ]:
defence_yards_df.head()

In [ ]:
personal_yards

In [ ]:
# average_yards関数で作った３つを、元のデータとmerge
def merge_lastYearAv(df_play, team_yards_df, defence_yards_df, personal_yards, year):
    last_year = year-1
    df = team_yards_df[[last_year]]
    df.columns = ["team_yards_av"]
    df_year = df_play[df_play["Season"]==year].merge(df, on=["Team","PlayerTeamAbbr"], how="left")

    df = defence_yards_df[[last_year]]
    df.columns = ["def_yards_av"]
    df_year = df_year.merge(df, on=["Team","DefenceTeamAbbr"], how="left")
    
    df_year = df_year.merge(
        personal_yards[last_year].rename(columns={'mean': 'player_yards_av', 'std': 'player_yards_std',
                                                 'count': 'player_yards_count'}), 
        on="NflId", how="left")   
    
    return df_year

In [ ]:
# average_yards関数で作った３つを、元のデータとmerge
def merge_anotherYearAv(df_play, team_yards_df, defence_yards_df, personal_yards):
    df = team_yards_df[[2017]]
    df.columns = ["team_yards_av"]
    df_2018 = df_play[df_play["Season"]==2018].merge(df, on=["Team","PlayerTeamAbbr"], how="left")

    df = defence_yards_df[[2017]]
    df.columns = ["def_yards_av"]
    df_2018 = df_2018.merge(df, on=["Team","DefenceTeamAbbr"], how="left")
    
    df_2018 = df_2018.merge(
        personal_yards[2017].rename(columns={'mean': 'player_yards_av', 'std': 'player_yards_std',
                                                 'count': 'player_yards_count'}), 
        on="NflId", how="left") 
    
    
    df = team_yards_df[[2018]]
    df.columns = ["team_yards_av"]
    df_2017 = df_play[df_play["Season"]==2017].merge(df, on=["Team","PlayerTeamAbbr"], how="left")

    df = defence_yards_df[[2018]]
    df.columns = ["def_yards_av"]
    df_2017 = df_2017.merge(df, on=["Team","DefenceTeamAbbr"], how="left")
    
    df_2017 = df_2017.merge(
        personal_yards[2018].rename(columns={'mean': 'player_yards_av', 'std': 'player_yards_std',
                                                 'count': 'player_yards_count'}), 
        on="NflId", how="left")  
    
    return pd.concat([df_2017, df_2018])

In [ ]:
def get_team_score(df_play):
    # 攻撃チームの得点
    df_play.loc[df_play["Team"]=="home", "rusherTeamScore"] = df_play["HomeScoreBeforePlay"]
    df_play.loc[df_play["Team"]=="away", "rusherTeamScore"] = df_play["VisitorScoreBeforePlay"]

    # 守備チームの得点
    df_play.loc[df_play["Team"]=="home", "defenceTeamScore"] = df_play["VisitorScoreBeforePlay"]
    df_play.loc[df_play["Team"]=="away", "defenceTeamScore"] = df_play["HomeScoreBeforePlay"]

    # 得点差
    df_play.loc[:, "diffScore"] = df_play["rusherTeamScore"] - df_play["defenceTeamScore"]
    
    return df_play

In [ ]:
df_play = get_team_score(df_play)

In [ ]:
def average_personal_data(df_play, df_all):
    # 攻撃,守備チーム平均 体重, 身長, S, A（PlayIdがキー）
    offence_av = df_all.loc[df_all["offence"]==1, ["PlayerHeight", "PlayerWeight", "S", "A", "PlayId"]].groupby("PlayId").mean()
    defence_av = df_all.loc[df_all["offence"]==0, ["PlayerHeight", "PlayerWeight", "S", "A", "PlayId"]].groupby("PlayId").mean()
    offence_av.columns = ['PlayerHeight_offence', 'PlayerWeight_offence', 'S_offence', 'A_offence']
    defence_av.columns = ['PlayerHeight_defence', 'PlayerWeight_defence', 'S_defence', 'A_defence']

    df_play = df_play.merge(offence_av, on="PlayId", how="left").merge(defence_av, on="PlayId", how="left")

    return df_play

In [ ]:
df_play = average_personal_data(df_play, df_all)

In [ ]:
def add_last_yards(df_play):
    # タッチダウンまで何ヤード
    df_play["yardsToTouchdown"] = df_play["YardLine"]
    df_play.loc[ df_play["PossessionTeam"] == df_play["FieldPosition"], "yardsToTouchdown"
                        ] = 100-df_play["YardLine"]
    return df_play
    

In [ ]:
df_play = add_last_yards(df_play)

In [ ]:
# 別年度の平均を結合
df_year = merge_anotherYearAv(df_play, team_yards_df, defence_yards_df, personal_yards)

# 使うデータを選ぶ

In [ ]:
# 欠測確認
df_year.isnull().sum().sort_values().tail(10)

In [ ]:
def select_columns(df):
    # 欠測あり列のうち、使用しないもの
    df_u = df.drop(["FieldPosition", "StadiumType", "GameWeather", "WindDirection"], axis=1)
    # Humidity、Temperature, WindSpeed　の欠測は平均で埋める
#    mean_cols = ["Humidity", "Temperature", "WindSpeed"]
#    df_u[mean_cols].fillna(df_u[mean_cols].mean(), inplace=True)
    df_u = df_u.fillna(df_u.mean())
    
#    df_use.loc[:, "Yards_class"] = df_use["Yards"]
#    df_use.loc[df_use["Yards"]>=20, "Yards_class"] = 20
#    df_use.loc[df_use["Yards"]<=-5, "Yards_class"] = -5
#'Yards' 'HomeTeamAbbr_x' 'HomeTeamAbbr_y' 'over20' 'touchdown'
        
    df_X = df_u.drop([ ## 文字列
                        "Team", "DisplayName", "PossessionTeam", 
                        "OffensePersonnel", "DefensePersonnel", "PlayDirection", 
                        "PlayerBirthDate", "PlayerCollegeName", 
                        "Position", "HomeTeamAbbr", "VisitorTeamAbbr", 
                        "Stadium", "Location", "Turf", "PlayerTeamAbbr", "DefenceTeamAbbr", 
                        # 時刻or時間
                        "GameClock", "TimeHandoff", "TimeSnap", 
                       # キー、番号
                       "GameId", "PlayId", "NflId", "JerseyNumber", "NflIdRusher"
                       ], axis=1)
    


    
    return df_X

def drop_columns(df_X):
# 効いていない特徴量を落とす
    df_X = df_X.drop(['Temperature', 'Humidity', 'diffScore', 'VisitorScoreBeforePlay',
       'PlayerHeight', 'PlayerHeight_defence', 'WindSpeed', 'Week', 'DB',
       'Down', 'DL', 'Quarter', 'LB', 'OffenseFormation_I_FORM', 'TE',
       'OffenseFormation_SHOTGUN', 'OffenseFormation_SINGLEBACK',
       'OffenseFormation_EMPTY', 'stadiumtype', 'OffenseFormation_JUMBO', 'RB',
       'OffenseFormation_PISTOL', 'weather', 'OffenseFormation_WILDCAT'
       ], axis=1)
    
    return df_X



In [ ]:
# pd.set_option("display.max_rows", 100)

In [ ]:
df_X = select_columns(df_year)
train_df=df_X.iloc[:0,:]
df_X = drop_columns(df_X)

df_y = df_X["Yards"]
df_X = df_X.drop(["Yards"], axis=1)

# Yard数の分類モデルとして学習

In [ ]:
# 分類で使うヤード数の最大最小
max_yards = 25
min_yards = -5

In [ ]:
print( len(df_y.loc[df_y>=max_yards]) )
print( len(df_y.loc[df_y<=min_yards]) )

In [ ]:
#df_use.loc[:, "Yards_class"] = df_use["Yards"]
df_y.loc[df_y>=max_yards] = max_yards
df_y.loc[df_y<=min_yards] = min_yards

In [ ]:
df_y.unique()

In [ ]:
# from sklearn.ensemble import RandomForestRegressor
from tqdm import tqdm
from sklearn.ensemble import RandomForestClassifier as classifier
from sklearn.model_selection import train_test_split


### パラメータチューニング

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:

# GridSearchCV によるチューニング
search_params = {
    'n_estimators'      : [100],
    'max_features'      : [10, 30], #,'auto'
    #'random_state'      : [1],
    #'n_jobs'            : [1],
    'min_samples_split' : [10, 20, 30],
    'max_depth'         : [3, 5, 10]
}
 
gsr = GridSearchCV(
    classifier(),
    search_params,
    cv = 3,
    n_jobs = -1,
    verbose=True
)


In [ ]:
# gsr.fit(df_X, df_y)

In [ ]:
'''
search_params = {
    'n_estimators'      : [100],
    'max_features'      : [3,10,30], #,'auto'
    #'random_state'      : [1],
    #'n_jobs'            : [1],
    'min_samples_split' : [10, 30],
    'max_depth'         : [3, 10, 20]
{'max_depth': 10, 'max_features': 30, 'min_samples_split': 30, 'n_estimators': 100}　0.1479300190542179
{'max_features': 30, 'min_samples_split': 10, 'max_depth': 3}　0.011617082679846237
'''

In [ ]:
'''
search_params = {
    'n_estimators'      : [100],
    'max_features'      : [30], #,'auto'
    #'random_state'      : [1],
    #'n_jobs'            : [1],
    'min_samples_split' : [10, 20, 30],
    'max_depth'         : [3, 5, 10, 15]
}
{'max_features': 30, 'min_samples_split': 30, 'max_depth': 3} 0.011455803585085578
'''

In [ ]:
# feature_importance_ が低いものを除外後
'''
search_params = {
    'n_estimators'      : [100],
    'max_features'      : [10, 30], #,'auto'
    #'random_state'      : [1],
    #'n_jobs'            : [1],
    'min_samples_split' : [10, 20, 30],
    'max_depth'         : [3, 5, 10]
}
0.011924821549978504
{'max_features': 10, 'min_samples_split': 10, 'max_depth': 3}
'''

In [ ]:
#print(gsr.best_score_)
#print(gsr.best_estimator_)
#print(gsr.best_params_)

In [ ]:
# train_X, test_X, train_y, test_y = train_test_split(df_X, df_y, test_size=0.3)

In [ ]:
# clf = gsr.best_estimator_
# clf.fit(train_X, train_y)

In [ ]:
def calc_score(pred_y, test_y):
    pred_df = pd.DataFrame(pred_y)
    pred_df.columns = [ "Yards"+str(n) for n in range(min_yards,max_yards+1)]
    
    pred_df_all = pd.DataFrame(pred_df,columns=[ "Yards"+str(n) for n in range(-99,100)])
    pred_df_all.fillna(0, inplace=True)
    pred_df_all = pred_df_all.cumsum(axis=1)
#    features.loc[:, :"Yards-6"] = 0
#    features.loc[:, "Yards21":] = 1
    
    test_df = pd.DataFrame(np.array([(i >= test_y)*1 for i in range(-99,100)]).T,
                           columns=pred_df_all.columns)
    
    CRPS=((pred_df_all-test_df)**2).mean().mean()
    return CRPS

In [ ]:
'''
best_score = 1.0
for features in tqdm(search_params['max_features']):
    for min_samp in tqdm(search_params['min_samples_split']):
        for depth in search_params['max_depth']:
            scores = []
            for i in range(3):
                train_X, test_X, train_y, test_y = train_test_split(df_X, df_y, test_size=0.3)
                clf = classifier(n_estimators=100, max_depth=20, min_samples_split=10)
                clf.fit(train_X, train_y)
                
                pred_y = clf.predict_proba(test_X)
                score = calc_score(pred_y, test_y)
                scores.append(score)
            av_score = np.mean(scores)
            if av_score < best_score:
                best_score = av_score
                best_params = {'max_features':features, 'min_samples_split':min_samp, 'max_depth':depth}
'''                


In [ ]:
# print(best_score)
# print(best_params)

### パラメータチューニング終わり

In [ ]:
# best_params

In [ ]:
#clf = classifier(n_estimators=100, 
#                 max_features = best_params["max_features"], 
#                 max_depth = best_params["max_depth"], 
#                 min_samples_split = best_params["min_samples_split"], random_state=777)

In [ ]:
clf = classifier(n_estimators=100, 
                 max_features = 10, 
                 max_depth = 3, 
                 min_samples_split = 10, random_state=777)

In [ ]:
clf.fit(df_X, df_y)

In [ ]:
feat_imp = pd.DataFrame( clf.feature_importances_, index=df_X.columns )

In [ ]:
feat_imp.sort_values(0, ascending=False).index

In [ ]:
feat_imp.sort_values(0, ascending=False)

# submission data 作成

In [ ]:
from kaggle.competitions import nflrush

In [ ]:
env = nflrush.make_env()

In [ ]:
#iter_test = env.iter_test()

In [ ]:
means = df_X.mean()

In [ ]:
# train_df=df_X.iloc[:0,:]

In [ ]:
# stopstop

In [ ]:
import warnings
warnings.simplefilter('ignore')

In [ ]:
train_df = train_df.drop("Yards", axis=1)

In [ ]:
df_X.columns

In [ ]:
for (test_df, sample_prediction_df) in tqdm(env.iter_test()):
    df_a = preprocess(test_df)

    df_ = df_a.loc[df_a["NflId"]==df_a["NflIdRusher"], :]

    df_a["offence"] = 0
    df_a.loc[df_a["Position"].isin(offence_position), "offence"] = 1

    df_ = team_name(df_)

    df_ = get_team_score(df_)
    df_ = average_personal_data(df_, df_a)
    df_ = add_last_yards(df_)

    df_year = merge_lastYearAv(df_, team_yards_df, defence_yards_df, personal_yards, 2019)
    test_X = select_columns(df_year)

    # 無いカテゴリー変数を埋める
    test_X=pd.concat([train_df,test_X],sort=False)
    test_X = test_X.fillna(0)
    
    test_X = drop_columns(test_X)

    pred520 = clf.predict_proba(test_X)[0]
    pred100 = np.concatenate([ np.zeros(99+min_yards), pred520.cumsum(), np.ones(99-max_yards) ])
    sample_prediction_df.iloc[0] = pred100

    # cumsum で１を僅かに超える場合がある
    sample_prediction_df[sample_prediction_df>1.0] = 1.0
    
    env.predict(sample_prediction_df)
    
    

In [ ]:
clf.classes_

In [ ]:
# test_df["WindSpeed"]

In [ ]:
# re.split('[^0-9.]+', "6mph")

In [ ]:
env.write_submission_file()

In [ ]:
import os
print([filename for filename in os.listdir('/kaggle/working') if '.csv' in filename])

In [ ]:
owariowari

# テスト

In [ ]:
(test_df, sample_prediction_df) = next(env.iter_test())

In [ ]:
df_all = preprocess(test_df)

In [ ]:
df_ = df_all[df_all["NflId"]==df_all["NflIdRusher"]]

In [ ]:
df_all["offence"] = 0
df_all.loc[df_all["Position"].isin(offence_position), "offence"] = 1

In [ ]:
df_ = team_name(df_)
# df_ = count_position(df_)
# team_yards_df, defence_yards_df, personal_yards = average_yards(df_)

In [ ]:
df_ = get_team_score(df_)
df_ = average_personal_data(df_, df_all)
df_ = add_last_yards(df_)

In [ ]:
df_year = merge_lastYearAv(df_, team_yards_df, defence_yards_df, personal_yards, 2019)

In [ ]:
test_X = select_columns(df_year)

In [ ]:
test_X.columns

In [ ]:
test_X=pd.concat([train_df,test_X],sort=False)
test_X = test_X.fillna(0)

In [ ]:
pred_df = pd.DataFrame(clf.predict_proba(test_X), columns=[ "Yards"+str(i) for i in clf.classes_])
pred_df = pred_df.cumsum(axis=1)

In [ ]:
import warnings
warnings.simplefilter('ignore')

In [ ]:
df_a = preprocess(test_df)

df_ = df_a.loc[df_a["NflId"]==df_a["NflIdRusher"], :]

df_a["offence"] = 0
df_a.loc[df_a["Position"].isin(offence_position), "offence"] = 1

df_ = team_name(df_)

df_ = get_team_score(df_)
df_ = average_personal_data(df_, df_a)
df_ = add_last_yards(df_)

df_year = merge_lastYearAv(df_, team_yards_df, defence_yards_df, personal_yards, 2019)
test_X = select_columns(df_year)

# 無いカテゴリー変数を埋める
test_X=pd.concat([train_df,test_X],sort=False)
test_X = drop_columns(test_X)
test_X = test_X.fillna(0)

pred520 = clf.predict_proba(test_X)[0]
pred100 = np.concatenate([ np.zeros(99+min_yards), pred520.cumsum(), np.ones(99-max_yards) ])
sample_prediction_df.iloc[0] = pred100
#pred_df = pd.DataFrame(clf.predict_proba(test_X), columns=[ "Yards"+str(i) for i in clf.classes_])
#pred_df = pred_df.cumsum(axis=1)

#sample_prediction_df.iloc[0, 95:121] = pred_df.iloc[0, :]
#sample_prediction_df.loc[:, :"Yards-6"] = 0
#sample_prediction_df.loc[:, "Yards21":] = 1

In [ ]:
sample_prediction_df

In [ ]:
sample_prediction_df.loc[:, "Yards-7":]